In [1]:
import pandas as pd
import numpy as np

import networkx as nx
from networkx.algorithms.community import k_clique_communities
import random as rd
rd.seed(42)

import warnings
warnings.filterwarnings('ignore')

## Random Walk Sampling Method 2
### Sample 2

In [2]:
G_rwm2_2 = nx.read_gml(r'C:\Users\sarah\Documents\UNI\Masters\Study\Term_3\Master Project\Samples\Random Walk Sampling\G_rw2_2.gml')

In [3]:
# Confirm that Graph is Directed
nx.is_directed(G_rwm2_2)

True

In [4]:
# Display Size of the Graph
print(nx.info(G_rwm2_2))

MultiDiGraph with 50000 nodes and 132346 edges


Confirmed that the graph meets the criterea of having more edges than nodes and maintained the directed structure.

In [5]:
# List all nodes in the sample
n_rwm2_2 = list(G_rwm2_2.nodes)

In [6]:
# Get node properties from data exploration exercise for comparison
node_properties = pd.read_csv(r'C:\Users\sarah\Documents\UNI\Masters\Study\Term_3\Master Project\node_properties_cl.csv')
node_properties[['degree', 'in_degree', 'out_degree', 'eigen_centrality']].describe()

,degree,in_degree,out_degree,eigen_centrality
count,4.666873e+06,4.666873e+06,4.666873e+06,4.666873e+06
mean,4.494298e+00,2.247149e+00,2.247149e+00,3.970074e-05
std,3.935922e+01,1.998412e+01,2.868249e+01,4.611943e-04
min,1.000000e+00,0.000000e+00,0.000000e+00,1.193322e-21
25%,1.000000e+00,0.000000e+00,0.000000e+00,1.193322e-21
50%,2.000000e+00,1.000000e+00,1.000000e+00,4.892620e-20
75%,3.000000e+00,1.000000e+00,2.000000e+00,1.517475e-14
max,2.791700e+04,1.211600e+04,2.533500e+04,1.642653e-01


In [7]:
rwm2_2_df = nx.to_pandas_edgelist(G_rwm2_2) 
rwm2_2_df.head()

,source,target,weight
0,1KXtmUX7963p1Nfqi2fgy77cRJoH65oron,12FpuP9pPNtCuA7ipz9UHH7G54Qm7RUFvn,1.0
1,1KXtmUX7963p1Nfqi2fgy77cRJoH65oron,1Mux24bnmLjZex2t7XvrmdXvqorkpvvU1s,1.0
2,13fHxjuQmErKUBryb2H3LPZvnXHhiacBHs,1ACMU68jxm7DEXzTcuks1zDK1P2FtgStEs,1.0
3,1CKo2GjEiKwaLcmr6bm2iX1i92Ex1fuuKy,15GnaeRGGFexJMUNZDquXddKkGNb3tiZuB,1.0
4,1CKo2GjEiKwaLcmr6bm2iX1i92Ex1fuuKy,1CK3vHiG661N7goC9e5EYpYczmhxsRNmgc,1.0


In [8]:
# Number of unique input addresses
rwm2_2_df['source'].nunique()

32078

In [9]:
# Number of unique output addresses
rwm2_2_df['target'].nunique()

31310

We observe relative balance between input and output addresses, with slightly more input addresses. In the full dataset there is also relative balance, but there were more output addresses. 

In [10]:
rwm2_2_df.describe()

,weight
count,132346.000000
mean,0.778339
std,0.587455
min,-99.311045
25%,0.527239
50%,1.000000
75%,1.000000
max,52.821905


The weight variable shows that the distribution of the weighting (amount*0.5 + timestamp*0.5) is similar to that of overall transaction amounts.

In [12]:
# Subset by fraudulent/high-risk nodes
fr_nodes = node_properties[node_properties['fraud_flag'] == 1]
fr_nodes = fr_nodes['address'].tolist()
print('Total number of fraudulent/high-risk nodes : ', len(fr_nodes))

# Check for number of fraudulent/high-risk addresses in the sample
print('Number of fraudulent/high-risk nodes in sample : ', len(set(n_rwm2_2).intersection(fr_nodes)))

Total number of fraudulent/high-risk nodes :  4563
Number of fraudulent/high-risk nodes in sample :  295


These numbers suggest that random selection of nodes worsens the class imbalance problem, but when considering the counts relative to the size of the network the issue is less pronounced.

Full Network: 0.11% 

Sample: 0.59%

## Node Properties

In [13]:
# Get Properties

address = [node for (node, val) in G_rwm2_2.degree()]
degree = [val for (node, val) in G_rwm2_2.degree()]
in_degree = [val for (node, val) in G_rwm2_2.in_degree()]
out_degree = [val for (node, val) in G_rwm2_2.out_degree()]
#eigen_centrality = nx.eigenvector_centrality(G_rwm2_2) # NetworkXNotImplemented: not implemented for multigraph type
#eigen_centrality = [eigen_centrality[node] for node in eigen_centrality]

In [14]:
# Add Properties to DataFrame

nodes = pd.DataFrame()
nodes['address'] = address
nodes['degree'] = degree
nodes['in_degree'] = in_degree
nodes['out_degree'] = out_degree
#nodes['eigen_centrality'] = eigen_centrality

In [15]:
# Add fraud flag

nodes = nodes.assign(**dict.fromkeys(['fraud_flag'], 0))

for i in fr_nodes:
    nodes.loc[nodes.address == i, 'fraud_flag'] = 1
    
nodes.head()

,address,degree,in_degree,out_degree,fraud_flag
0,1KXtmUX7963p1Nfqi2fgy77cRJoH65oron,2,0,2,0
1,13fHxjuQmErKUBryb2H3LPZvnXHhiacBHs,2,1,1,0
2,1CKo2GjEiKwaLcmr6bm2iX1i92Ex1fuuKy,2,0,2,0
3,13QgkvueEWcniBkV3jr1TWfeeY3Nz5H9QP,1,1,0,0
4,1HLwvf5m8YTNzYByWxCULG6wpJ7VfHz4Nq,5,1,4,0


In [16]:
# Describe sample properties overall

nodes[['degree', 'in_degree', 'out_degree']].describe()

,degree,in_degree,out_degree
count,50000.000000,50000.000000,50000.000000
mean,5.293840,2.646920,2.646920
std,38.583413,17.132168,24.795395
min,1.000000,0.000000,0.000000
25%,1.000000,0.000000,0.000000
50%,2.000000,1.000000,1.000000
75%,4.000000,2.000000,2.000000
max,4172.000000,1507.000000,2672.000000


In [17]:
# Describe Properties of Fraudulent Nodes

nodes_fraud = nodes[nodes['fraud_flag'] == 1]
nodes_fraud[['degree', 'in_degree', 'out_degree']].describe()

,degree,in_degree,out_degree
count,295.000000,295.000000,295.000000
mean,124.644068,51.372881,73.271186
std,450.007149,181.069226,293.611018
min,1.000000,0.000000,0.000000
25%,5.000000,2.000000,1.000000
50%,20.000000,9.000000,6.000000
75%,76.500000,37.000000,32.500000
max,4172.000000,1507.000000,2672.000000


In [18]:
# Describe Properties of Non-Fraudulent Nodes

nodes_licit = nodes[nodes['fraud_flag'] == 0]
nodes_licit[['degree', 'in_degree', 'out_degree']].describe()

,degree,in_degree,out_degree
count,49705.000000,49705.000000,49705.000000
mean,4.585494,2.357731,2.227764
std,14.650652,9.335316,8.874871
min,1.000000,0.000000,0.000000
25%,1.000000,0.000000,0.000000
50%,2.000000,1.000000,1.000000
75%,4.000000,2.000000,2.000000
max,846.000000,837.000000,644.000000


For all 3 cases the degree distribution properties are similar to the overall network.

## Graph Properties

In [19]:
# Density
nx.density(G_rwm2_2) # Connectivity remains very low

5.293945878917578e-05

In [20]:
# Communities - k-Cliques

com_generator_rwm2_2 = k_clique_communities(G_rwm2_2.to_undirected(), k=10)
com_rwm2_2 = next(com_generator_rwm2_2)
print('The number of communities in the sample : ' + str(len(com_rwm2_2)))

The number of communities in the sample : 98


The sample has a large variety of community structures. (Note: Giant Component = 107)